In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import pandas as pd

C:\Users\Yuki\OneDrive - The Pennsylvania State University\EPiC2023\with_dynamic_features\mvts_transformer\ts_transformer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # normalize image to [-1, 1]

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False)

# 10 classes in total
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck') 

Files already downloaded and verified
Files already downloaded and verified


In [16]:
# load test data (note that the data has been transformed already)
test_images = torch.load('test_image.pt')

In [17]:
# specify the GPU device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [18]:
import torchvision.models as models

In [19]:
#to try ResNet models with different number of layers
def create_resnet(num_layers, dropout_rate):
    if num_layers == 18:
        net = models.resnet18(pretrained=False,num_classes=10).to(device)
    elif num_layers == 34:
        net = models.resnet34(pretrained=False,num_classes=10).to(device)
    elif num_layers == 50:
        net = models.resnet50(pretrained=False,num_classes=10).to(device)
    else:
        raise ValueError("Unsupported number of layers")

    num_features = net.fc.in_features
    net.fc = nn.Sequential(
        nn.Dropout(dropout_rate),
        nn.Linear(num_features, 10)
    )
    return net




In [16]:
#this is the final model for the kaggle submission
# num_layers = 18, dropout_rate = 0.29578761633364997
#net = models.resnet18(pretrained=False, num_classes=10).to(device)
net = create_resnet(num_layers=18, dropout_rate=0.29578761633364997).to(device)

In [7]:
import torch.optim as optim
import torch.nn as nn
import optuna

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0007836827942082852, momentum=0.9882941058204819, weight_decay=9.116296060594481e-05)

#results from two attemps with hyperparameter tuning using optuna
#Best trial: score 66.76, params {'lr': 0.0008323950770691731, 'momentum': 0.9659751984631002, 'weight_decay': 0.0001880406194490467}  layers = 18, drop out rate = 0.1
#Best trial: score 69.5, params {'lr': 0.0007836827942082852, 'momentum': 0.9882941058204819, 'weight_decay': 9.116296060594481e-05, 'num_layers': 18, 'dropout_rate': 0.29578761633364997}

best_acc = 0.0  # track the best accuracy

# Training loop
for epoch in range(20):  # loop over the dataset multiple times (e.g., 20 epochs)
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data[0].to(device), data[1].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1} loss: {running_loss / len(trainloader)}")
    
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    print(f"Epoch {epoch+1} Accuracy: {acc}%")

    # Check if this model has the best accuracy
    if acc > best_acc:
        best_acc = acc
        torch.save(net.state_dict(), 'best_model.pth')  # save the best model
        
print('Finished Training')

Epoch 1 loss: 1.7304518082562614
Epoch 1 Accuracy: 47.05%
Epoch 2 loss: 1.3162072363411983
Epoch 2 Accuracy: 54.95%
Epoch 3 loss: 1.1204775876706214
Epoch 3 Accuracy: 58.86%
Epoch 4 loss: 0.9777314966292028
Epoch 4 Accuracy: 62.31%
Epoch 5 loss: 0.8298478890258028
Epoch 5 Accuracy: 63.63%
Epoch 6 loss: 0.710275267319911
Epoch 6 Accuracy: 65.37%
Epoch 7 loss: 0.6033244251137804
Epoch 7 Accuracy: 66.84%
Epoch 8 loss: 0.49758579900197664
Epoch 8 Accuracy: 66.22%
Epoch 9 loss: 0.43003208790441305
Epoch 9 Accuracy: 66.63%
Epoch 10 loss: 0.37230549676491476
Epoch 10 Accuracy: 66.83%
Epoch 11 loss: 0.3047101519373067
Epoch 11 Accuracy: 67.08%
Epoch 12 loss: 0.2545683096398783
Epoch 12 Accuracy: 67.95%
Epoch 13 loss: 0.21307590590489794
Epoch 13 Accuracy: 67.55%
Epoch 14 loss: 0.1854083371894134
Epoch 14 Accuracy: 68.43%
Epoch 15 loss: 0.1505614503970384
Epoch 15 Accuracy: 67.75%
Epoch 16 loss: 0.14034077464162237
Epoch 16 Accuracy: 68.41%
Epoch 17 loss: 0.12408013030162553
Epoch 17 Accuracy: 

In [18]:
import numpy as np

In [19]:
#make predictions to the kaggle test dataset
#model = models.resnet18(pretrained=False, num_classes=10).to(device)
model = create_resnet(num_layers=18, dropout_rate=0.29578761633364997).to(device)
model.load_state_dict(torch.load('best_model.pth'))
model.eval()  # Set the model to evaluation mode

# Use the model for predictions
with torch.no_grad():
    test_images = test_images.to(device)
    output = model(test_images)
    _, predicted = torch.max(output.data, 1)
    predicted = np.array([classes[i] for i in predicted])
    
print(predicted)   
    
submission = pd.DataFrame()
submission['label'] = predicted
submission.to_csv("submission1026.csv", index=True, index_label='id')
submission

['cat' 'dog' 'frog' ... 'plane' 'bird' 'bird']


label
0       cat
1       dog
2      frog
3      bird
4      ship
...     ...
4995   deer
4996   ship
4997  plane
4998   bird
4999   bird

[5000 rows x 1 columns]

In [20]:
###########with hyperparameter tuning#################
import optuna
import torch
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

In [21]:
 def objective(trial):
    lr = trial.suggest_float('lr', 1e-5, 1e-3, log=True)
    momentum = trial.suggest_float('momentum', 0.5, 0.99)
    weight_decay = trial.suggest_float('weight_decay', 1e-5, 1e-4, log=True)
    num_layers = trial.suggest_categorical('num_layers', [18, 34, 50])
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)

    net = create_resnet(num_layers, dropout_rate).to(device) 
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

    best_acc = 0.0

    for epoch in range(15):  #after several attempts, it seems like the loss become stable after 10 epochs
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        correct = 0
        total = 0
        with torch.no_grad():
            for data in testloader:
                images, labels = data[0].to(device), data[1].to(device)
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        acc = 100 * correct / total
        print(f"Epoch {epoch+1} Accuracy: {acc}%")
        if acc > best_acc:
            best_acc = acc

    return best_acc  

# optimization
study = optuna.create_study(direction='maximize')  
study.optimize(objective, n_trials=1)  #for the kaggle project, the tuning was based on 50 trials.

# Print best results
print(f'Best trial: score {study.best_value}, params {study.best_params}')

[I 2023-10-26 19:46:20,304] A new study created in memory with name: no-name-a8859716-3a90-45b4-83a6-5ec1e1fc5bbe


Epoch 1 Accuracy: 35.55%
Epoch 2 Accuracy: 43.59%
Epoch 3 Accuracy: 46.43%
Epoch 4 Accuracy: 48.83%
Epoch 5 Accuracy: 51.35%
Epoch 6 Accuracy: 52.49%
Epoch 7 Accuracy: 53.54%
Epoch 8 Accuracy: 53.23%
Epoch 9 Accuracy: 54.05%
Epoch 10 Accuracy: 53.94%
Epoch 11 Accuracy: 54.73%
Epoch 12 Accuracy: 53.81%
Epoch 13 Accuracy: 53.57%
Epoch 14 Accuracy: 53.87%


[I 2023-10-26 19:52:13,100] Trial 0 finished with value: 54.73 and parameters: {'lr': 0.0002731547816681979, 'momentum': 0.9361874929587692, 'weight_decay': 1.591334024066517e-05, 'num_layers': 34, 'dropout_rate': 0.3432538481627849}. Best is trial 0 with value: 54.73.


Epoch 15 Accuracy: 54.04%
Best trial: score 54.73, params {'lr': 0.0002731547816681979, 'momentum': 0.9361874929587692, 'weight_decay': 1.591334024066517e-05, 'num_layers': 34, 'dropout_rate': 0.3432538481627849}
